In [147]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

In [148]:
# read in data from July, 28th, 2012
trajectories28 = pd.read_csv("data/TrajDataV2_20120728.txt",  
                             delimiter="\t", 
                             names=('flight', 'date', 'wind', 'time', 'speed', 'altitude','latitude', 'longitude', 'nan'),
                             usecols=('flight', 'date', 'wind', 'time', 'speed', 'altitude','latitude', 'longitude')
                            )
# read in data from July, 29th, 2012
trajectories29 = pd.read_csv("data/TrajDataV2_20120729.txt",  
                             delimiter="\t", 
                             names=('flight', 'date', 'wind', 'time', 'speed', 'altitude','latitude', 'longitude', 'nan'),
                             usecols=('flight', 'date', 'wind', 'time', 'speed', 'altitude','latitude', 'longitude')
                            )
trajectories28.head()

,flight,date,wind,time,speed,altitude,latitude,longitude
0,845,28,0,1,470.31621,39000,40.692500,-74.168667
1,845,28,0,2,470.31621,39000,40.784686,-74.019193
2,845,28,0,3,470.31621,39000,40.877281,-73.868705
3,845,28,0,4,470.31621,39000,40.969822,-73.717979
4,845,28,0,5,470.31621,39000,41.062186,-73.566848


In [149]:
# concatenate data from both days
#trajectories = pd.concat([trajectories28, trajectories29])
trajectories = trajectories29

In [150]:
# add consecutive flight index to the data
flightNames = trajectories['flight'].unique()
trajectories['flightIndex'] = trajectories['flight'].map(lambda x: np.where(flightNames==x)[0][0])
trajectories.head()

,flight,date,wind,time,speed,altitude,latitude,longitude,flightIndex
0,805,29,0,2,465.28117,33000,40.692500,-74.168667,0
1,805,29,0,3,465.28117,33000,40.785863,-74.013012,0
2,805,29,0,4,465.28117,33000,40.878762,-73.856473,0
3,805,29,0,5,465.28117,33000,40.971273,-73.699485,0
4,805,29,0,6,465.28117,33000,41.063590,-73.542087,0


In [151]:
# set consecutive flight index as dataset index
trajectories = trajectories.set_index('flightIndex')
trajectories.head()

,flight,date,wind,time,speed,altitude,latitude,longitude
flightIndex,,,,,,,,
0,805,29,0,2,465.28117,33000,40.692500,-74.168667
0,805,29,0,3,465.28117,33000,40.785863,-74.013012
0,805,29,0,4,465.28117,33000,40.878762,-73.856473
0,805,29,0,5,465.28117,33000,40.971273,-73.699485
0,805,29,0,6,465.28117,33000,41.063590,-73.542087


In [152]:
trajectories.time.plot(marker='o', linestyle='o')
plt.show()

In [153]:
def plotTrajectories(trajectories):
    # Create a figure of size (i.e. pretty big)
    fig = plt.figure(figsize=(20,10))

    # Create a map, using the Gall–Peters projection, 
    map = Basemap(projection='gall', 
                  # with low resolution,
                  resolution = 'l', 
                  # And threshold 100000
                  area_thresh = 100000.0,
                  # Centered at 0,0 (i.e null island)
                  lat_0=0, lon_0=0)

    # Draw the coastlines on the map
    map.drawcoastlines()

    # Draw country borders on the map
    map.drawcountries()

    # Fill the land with grey
    map.fillcontinents(color = '#888888')

    # Draw the map boundaries
    map.drawmapboundary(fill_color='#f4f4f4')

    # Define our longitude and latitude points
    # We have to use .values because of a wierd bug when passing pandas data
    # to basemap.
    x,y = map(trajectories['longitude'].values, trajectories['latitude'].values)

    # Plot them using round markers of size 6
    map.plot(x, y, 'b', markersize=6, marker='o', linestyle='o')

    # Show the map
    plt.show()

In [131]:
plotTrajectories(trajectories[trajectories.index < 100])

In [154]:
pointConflicts = pd.read_csv("pointConflicts.dat",  
                             delimiter=",", header=1, names=['conflict', 'flight1', 'flight2', 'lat1', 'lon1', 'time1', 'lat2', 'lon2', 'time2'],
                             index_col=0
                            )

In [155]:
pointConflicts.head()

,flight1,flight2,lat1,lon1,time1,lat2,lon2,time2
conflict,,,,,,,,
1,148,126,26.499067,-79.911522,154,26.502781,-79.893341,195
2,148,126,26.617542,-79.848465,155,26.621145,-79.827339,196
3,148,126,26.735987,-79.785278,156,26.739475,-79.761192,197
4,126,148,26.384378,-79.959198,194,26.380560,-79.974449,153
5,126,148,26.502781,-79.893341,195,26.499067,-79.911522,154


In [156]:
pointConflictTrajectories1 = pointConflicts[['lat1','lon1']]
pointConflictTrajectories1.columns = ['latitude', 'longitude']
pointConflictTrajectories2 = pointConflicts[['lat2','lon2']]
pointConflictTrajectories2.columns = ['latitude', 'longitude']
pointConflictTrajectories1.head()

,latitude,longitude
conflict,,
1,26.499067,-79.911522
2,26.617542,-79.848465
3,26.735987,-79.785278
4,26.384378,-79.959198
5,26.502781,-79.893341


In [157]:
def plotTrajectoriesAndPointConflicts(trajectories, pointConflicts1, pointConflicts2):
    # Create a figure of size (i.e. pretty big)
    fig = plt.figure(figsize=(20,10))

    # Create a map, using the Gall–Peters projection, 
    map = Basemap(projection='gall', 
                  # with low resolution,
                  resolution = 'l', 
                  # And threshold 100000
                  area_thresh = 100000.0,
                  # Centered at 0,0 (i.e null island)
                  lat_0=0, lon_0=0)

    # Draw the coastlines on the map
    map.drawcoastlines()

    # Draw country borders on the map
    map.drawcountries()

    # Fill the land with grey
    map.fillcontinents(color = '#888888')

    # Draw the map boundaries
    map.drawmapboundary(fill_color='#f4f4f4')

    # Define our longitude and latitude points
    # We have to use .values because of a wierd bug when passing pandas data
    # to basemap.
    x,y = map(trajectories['longitude'].values, trajectories['latitude'].values)

    # Plot them using round markers of size 6
    map.plot(x, y, 'b', markersize=2, marker='+')
    x,y = map(pointConflicts1['longitude'].values, pointConflicts1['latitude'].values)
    map.plot(x, y, 'r', markersize=6, marker='<', linestyle='o')
    x,y = map(pointConflicts2['longitude'].values, pointConflicts2['latitude'].values)
    map.plot(x, y, 'g', markersize=6, marker='>', linestyle='o')

    # Show the map
    plt.show()

In [159]:
plotTrajectoriesAndPointConflicts(trajectories, pointConflictTrajectories1, pointConflictTrajectories2)

In [169]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.patches as mpatches

def add_arrow_to_line2D(
    axes, line, arrow_locs=[0.5],
    arrowstyle='-|>', arrowsize=1, transform=None):
    """
    Add arrows to a matplotlib.lines.Line2D at selected locations.

    Parameters:
    -----------
    axes: 
    line: list of 1 Line2D obbject as returned by plot command
    arrow_locs: list of locations where to insert arrows, % of total length
    arrowstyle: style of the arrow
    arrowsize: size of the arrow
    transform: a matplotlib transform instance, default to data coordinates

    Returns:
    --------
    arrows: list of arrows
    """
    if (not(isinstance(line, list)) or not(isinstance(line[0], 
                                           mlines.Line2D))):
        raise ValueError("expected a matplotlib.lines.Line2D object")
    x, y = line[0].get_xdata(), line[0].get_ydata()

    arrow_kw = dict(arrowstyle=arrowstyle, mutation_scale=10 * arrowsize)

    color = line[0].get_color()
    use_multicolor_lines = isinstance(color, np.ndarray)
    if use_multicolor_lines:
        raise NotImplementedError("multicolor lines not supported")
    else:
        arrow_kw['color'] = color

    linewidth = line[0].get_linewidth()
    if isinstance(linewidth, np.ndarray):
        raise NotImplementedError("multiwidth lines not supported")
    else:
        arrow_kw['linewidth'] = linewidth

    if transform is None:
        transform = axes.transData

    arrows = []
    for loc in arrow_locs:
        s = np.cumsum(np.sqrt(np.diff(x) ** 2 + np.diff(y) ** 2))
        n = np.searchsorted(s, s[-1] * loc)
        arrow_tail = (x[n], y[n])
        arrow_head = (np.mean(x[n:n + 2]), np.mean(y[n:n + 2]))
        p = mpatches.FancyArrowPatch(
            arrow_tail, arrow_head, transform=transform,
            **arrow_kw)
        axes.add_patch(p)
        arrows.append(p)
    return arrows

In [239]:
def plotTrajectoriesAndPointConflicts(trajectories, pointConflicts1, pointConflicts2, eastWest=False):
    # Create a figure of size (i.e. pretty big)
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(1, 1, 1)

    # Create a map, using the Gall–Peters projection, 
    map = Basemap(projection='gall', 
                  # with low resolution,
                  resolution = 'l', 
                  # And threshold 100000
                  area_thresh = 100000.0,
                  # Centered at 0,0 (i.e null island)
                  lat_0=0, lon_0=0)

    # Draw the coastlines on the map
    map.drawcoastlines()

    # Draw country borders on the map
    map.drawcountries()

    # Fill the land with grey
    map.fillcontinents(color = '#888888')

    # Draw the map boundaries
    map.drawmapboundary(fill_color='#f4f4f4')

    # Define our longitude and latitude points
    # We have to use .values because of a wierd bug when passing pandas data
    # to basemap.

    if eastWest:
        for flightIndex in set(trajectories.index):
            x,y = map(trajectories[trajectories.index == flightIndex]['longitude'].values, trajectories[trajectories.index == flightIndex]['latitude'].values)
            if x[1] > x[0]:
                map.plot(x, y, 'b', markersize=2, marker='+')
            else:
                map.plot(x, y, color='brown', markersize=2, marker='+')
    else:
        x,y = map(trajectories['longitude'].values, trajectories['latitude'].values)
        map.plot(x, y, 'b', markersize=2, marker='+')

    x,y = map(pointConflicts1['longitude'].values, pointConflicts1['latitude'].values)
    map.plot(x, y, 'r', markersize=6, marker='<', linestyle='o')
    x,y = map(pointConflicts2['longitude'].values, pointConflicts2['latitude'].values)
    map.plot(x, y, 'g', markersize=6, marker='>', linestyle='o')
    # Show the map
    plt.show()

In [240]:
plotTrajectoriesAndPointConflicts(trajectories, pointConflictTrajectories1, pointConflictTrajectories2, eastWest=True)